<a href="https://colab.research.google.com/github/heomanism/Bioinformatics1/blob/main/Free_Proj/Bioinformatics1_FreeProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#생물정보학 자유 연구 주제

## 1. Fig S3C 재현해보기
## 2. LIN28A binding motif predictor 만들기
## 3. 2차구조를 같이 고려한 LIN28A binding motif predictor 만들기

### 2021-20784 허민(Min Heo)

## Google Drive 마운트

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 환경 세팅

In [17]:
# !git clone https://github.com/hyeshik/colab-biolab.git
# !cd colab-biolab && bash tools/setup.sh
# exec(open('colab-biolab/tools/activate_conda.py').read())

In [18]:
#!conda install -y bedtools bioawk

In [3]:
%cd /content/drive/MyDrive/binfo1-work/

/content/drive/MyDrive/binfo1-work


In [19]:
#! ls -al

In [ ]:
## Lin28a를 포함하는 transcript 찾기 & 범위 알아보기
!cat gencode.gtf | awk '$3=="transcript" {print$0}' | grep -i lin28a

chr4	HAVANA	transcript	133730641	133746152	.	-	.	gene_id "ENSMUSG00000050966.10"; transcript_id "ENSMUST00000051674.3"; gene_type "protein_coding"; gene_name "Lin28a"; transcript_type "protein_coding"; transcript_name "Lin28a-201"; level 2; protein_id "ENSMUSP00000050488.3"; transcript_support_level "1"; mgi_id "MGI:1890546"; tag "basic"; tag "appris_principal_1"; tag "CCDS"; ccdsid "CCDS18761.1"; havana_gene "OTTMUSG00000010211.3"; havana_transcript "OTTMUST00000023733.1";
chr4	HAVANA	transcript	133733555	133739697	.	-	.	gene_id "ENSMUSG00000050966.10"; transcript_id "ENSMUST00000176113.8"; gene_type "protein_coding"; gene_name "Lin28a"; transcript_type "protein_coding"; transcript_name "Lin28a-202"; level 2; protein_id "ENSMUSP00000135254.2"; transcript_support_level "5"; mgi_id "MGI:1890546"; tag "mRNA_end_NF"; tag "cds_end_NF"; havana_gene "OTTMUSG00000010211.3"; havana_transcript "OTTMUST00000104013.2";
chr4	HAVANA	transcript	133733561	133736722	.	-	.	gene_id "ENSMUSG00000050966.1

In [ ]:
## 해당 좌표 범위로 CLIP-lin28a.bam 파일 만들기
!samtools view -b -o CLIP-lin28a.bam CLIP-35L33G.bam chr4:133730641-133746152 chr4:133733555-133739697 chr4:133733561-133736722 chr4:133735259-133745693
!samtools view CLIP-lin28a.bam | wc -l

5349


In [ ]:
! samtools sort CLIP-lin28a.bam -o Sorted_CLIP-lin28a.bam

In [ ]:
## samtools mpileup for watching read's distribution
!samtools mpileup Sorted_CLIP-lin28a.bam > CLIP-lin28a.pileup
!wc -l CLIP-lin28a.pileup

[mpileup] 1 samples in 1 input files
144525 CLIP-lin28a.pileup


In [ ]:
!head CLIP-lin28a.pileup

chr4	133655662	N	4	^It^It^It^Gt	I<I:
chr4	133655663	N	4	tttt	IFIB
chr4	133655664	N	4	tttt	IHIE
chr4	133655665	N	4	tttt	IDIE
chr4	133655666	N	4	cccc	IIID
chr4	133655667	N	4	cccc	IIIE
chr4	133655668	N	4	aaaa	IGIB
chr4	133655669	N	4	gggg	IIIG
chr4	133655670	N	4	<<<<	IGIG
chr4	133655671	N	4	<<<<	IGIG


In [ ]:
!awk '($2 >= 133730641 && $2 <= 133746152) || ($2 >= 133733555 && $2 <= 133739697) || ($2 >= 133733561 && $2 <= 133736722) || ($2 >= 133735259 && $2 <= 133745693)   { print $0; }' CLIP-lin28a.pileup > CLIP-lin28a-gene.pileup
!tail CLIP-lin28a-gene.pileup

chr4	133746143	N	48	<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<	IH=EHGIIH=EHGIIH=EHGIIH=EHGIIHIIDIHIIDIHIIDIHIID
chr4	133746144	N	48	<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<	IH=EHGIIH=EHGIIH=EHGIIH=EHGIIHIIDIHIIDIHIIDIHIID
chr4	133746145	N	48	<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<	IH=EHGIIH=EHGIIH=EHGIIH=EHGIIHIIDIHIIDIHIIDIHIID
chr4	133746146	N	48	<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<	IH=EHGIIH=EHGIIH=EHGIIH=EHGIIHIIDIHIIDIHIIDIHIID
chr4	133746147	N	48	<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<	IH=EHGIIH=EHGIIH=EHGIIH=EHGIIHIIDIHIIDIHIIDIHIID
chr4	133746148	N	48	<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<	IH=EHGIIH=EHGIIH=EHGIIH=EHGIIHIIDIHIIDIHIIDIHIID
chr4	133746149	N	48	<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<	IH=EHGIIH=EHGIIH=EHGIIH=EHGIIHIIDIHIIDIHIIDIHIID
chr4	133746150	N	48	<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<	IH=EHGIIH=EHGIIH=EHGIIH=EHGIIHIIDIHIIDIHIIDIHIID
chr4	133746151	N	48	<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

In [ ]:
import pandas as pd

lin28a_pileup = pd.read_csv('CLIP-lin28a-gene.pileup', sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals'])
lin28a_pileup

,chrom,pos,_ref,count,basereads,quals
0,chr4,133730641,N,49,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IGIGGIH=EHGIIH=EHGIIH=EHGIIH=EHGIHIDBHIDBHIDBHIDB
1,chr4,133730642,N,49,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IGIGGIH=EHGIIH=EHGIIH=EHGIIH=EHGIHIDBHIDBHIDBHIDB
2,chr4,133730643,N,49,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IGIGGIH=EHGIIH=EHGIIH=EHGIIH=EHGIHIDBHIDBHIDBHIDB
3,chr4,133730644,N,49,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IGIGGIH=EHGIIH=EHGIIH=EHGIIH=EHGIHIDBHIDBHIDBHIDB
4,chr4,133730645,N,49,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IGIGGIH=EHGIIH=EHGIIH=EHGIIH=EHGIHIDBHIDBHIDBHIDB
...,...,...,...,...,...,...
15507,chr4,133746148,N,48,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IH=EHGIIH=EHGIIH=EHGIIH=EHGIIHIIDIHIIDIHIIDIHIID
15508,chr4,133746149,N,48,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IH=EHGIIH=EHGIIH=EHGIIH=EHGIIHIIDIHIIDIHIIDIHIID
15509,chr4,133746150,N,48,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IH=EHGIIH=EHGIIH=EHGIIH=EHGIIHIIDIHIIDIHIIDIHIID
15510,chr4,133746151,N,48,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IH=EHGIIH=EHGIIH=EHGIIH=EHGIIHIIDIHIIDIHIIDIHIID


In [ ]:
import re
toremove = re.compile('[<>$*#]')  ## 이렇게 하는 것이 맞는게 아닐까?
toremove2= re.compile('\^.')  ## 이렇게 하는 것이 맞는게 아닐까?
lin28a_pileup['matches'] = lin28a_pileup['basereads'].apply(lambda x: toremove.sub('', x))
lin28a_pileup['matches'] = lin28a_pileup['matches'].apply(lambda x: toremove2.sub('', x))
lin28a_pileup

,chrom,pos,_ref,count,basereads,quals,matches
0,chr4,133730641,N,49,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IGIGGIH=EHGIIH=EHGIIH=EHGIIH=EHGIHIDBHIDBHIDBHIDB,
1,chr4,133730642,N,49,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IGIGGIH=EHGIIH=EHGIIH=EHGIIH=EHGIHIDBHIDBHIDBHIDB,
2,chr4,133730643,N,49,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IGIGGIH=EHGIIH=EHGIIH=EHGIIH=EHGIHIDBHIDBHIDBHIDB,
3,chr4,133730644,N,49,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IGIGGIH=EHGIIH=EHGIIH=EHGIIH=EHGIHIDBHIDBHIDBHIDB,
4,chr4,133730645,N,49,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IGIGGIH=EHGIIH=EHGIIH=EHGIIH=EHGIHIDBHIDBHIDBHIDB,
...,...,...,...,...,...,...,...
15507,chr4,133746148,N,48,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IH=EHGIIH=EHGIIH=EHGIIH=EHGIIHIIDIHIIDIHIIDIHIID,
15508,chr4,133746149,N,48,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IH=EHGIIH=EHGIIH=EHGIIH=EHGIIHIIDIHIIDIHIIDIHIID,
15509,chr4,133746150,N,48,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IH=EHGIIH=EHGIIH=EHGIIH=EHGIIHIIDIHIIDIHIIDIHIID,
15510,chr4,133746151,N,48,<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<,IH=EHGIIH=EHGIIH=EHGIIH=EHGIIHIIDIHIIDIHIIDIHIID,


In [ ]:
lin28a_pileup[['chrom', 'pos','_ref','matches']]

,chrom,pos,_ref,matches
0,chr4,133730641,N,
1,chr4,133730642,N,
2,chr4,133730643,N,
3,chr4,133730644,N,
4,chr4,133730645,N,
...,...,...,...,...
15507,chr4,133746148,N,
15508,chr4,133746149,N,
15509,chr4,133746150,N,
15510,chr4,133746151,N,


In [14]:
## Make Entropy calculation function for genome
import collections
 
from scipy.stats import entropy
  
def estimate_shannon_entropy(dna_sequence):
    bases = collections.Counter([tmp_base for tmp_base in dna_sequence])
    # define distribution
    dist = [x/sum(bases.values()) for x in bases.values()]
 
    # use scipy to calculate entropy
    entropy_value = entropy(dist, base=2)
 
    return entropy_value

In [20]:
#lin28a_pileup['entropy'] = lin28a_pileup['matches'].map(lambda x : estimate_shannon_entropy(x))
#lin28a_pileup['entropy']

In [21]:
# bedgraph_lin28a = lin28a_pileup[['chrom','pos','pos','entropy']].copy()
# bedgraph_lin28a.columns = ['chrom','chromStart','chromEnd','dataValue']

In [22]:
# bedgraph_lin28a.to_csv('lin28a.BedGraph', index=False,sep=' ') 

## ~ 06-03

### Transcriptome-wide level로 mpileup 파일 구한 후, read > 50으로 filtering

In [ ]:
## transcriptome wide level mpileup
#! samtools mpileup CLIP-35L33G.bam > CLIP-35L33G.pileup

[mpileup] 1 samples in 1 input files


In [7]:
! head CLIP-35L33G.pileup 
# chromosome position reference count basereads quals 

chr1	3056473	N	4	^!G^!G^!G^!G	HGGI
chr1	3056474	N	4	AAAA	HB@I
chr1	3056475	N	4	AAAA	GG?I
chr1	3056476	N	4	TTTT	HGGH
chr1	3056477	N	4	GGGG	HEGI
chr1	3056478	N	4	GGGG	GEEI
chr1	3056479	N	4	AAAA	EA=I
chr1	3056480	N	4	AAAA	@AEH
chr1	3056481	N	4	GGGG	GFFI
chr1	3056482	N	4	TTTT	G=BG


In [9]:
#! cat CLIP-35L33G.pileup | awk '$4 > 50{print$0}' > CLIP-35L33G_filtered.pileup

In [10]:
import pandas as pd

pileup_35L33G = pd.read_csv('CLIP-35L33G_filtered.pileup', sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals'])
pileup_35L33G

,chrom,pos,_ref,count,basereads,quals
0,chr1,3222722,N,124,CCCAAAAAAAAAAAAAAAAAAAAAAAAA^!A^!A^!A^!A^!A^!A...,HCGGHEIHHHIBDDIIIGIG>GIHGDHHDDIAEIEEBIGIBGG8GI...
1,chr1,3222723,N,146,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,E>HIHFIIHHIGDFIII<IHBIIHGDHEB?I@HIEIBIII;>G;GI...
2,chr1,3222724,N,155,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,H>GIGHHIHHDG:@IIIBIHDIIHG=HHDDG;IHHH;IIIDG?DIE...
3,chr1,3222725,N,157,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...,HGDEHHIIHHIG?DIIIEIGDIIHDBBH=DI;IIHBBIII4GD3GI...
4,chr1,3222726,N,156,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,HB:IHGIIGGIGDFHII@IFBHGHGBHGCBI@IIHGBIGI;>E@BI...
...,...,...,...,...,...,...
24696039,MU069435.1,1564,N,305,>>>>TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,DIDHGHIIGIGIEDIGIHIIG?FBGG@E=:A7G8@HHIHIIDDBII...
24696040,MU069435.1,1565,N,297,>>>>AAAAAAAAAAAA$AA$AA$A$AA$AA$A$AAAAAAAAAAAAA...,DIDHBHHIIIGICGIGIEIHDEFBGE8C=1?1G88IHIHIIDGBHI...
24696041,MU069435.1,1566,N,289,>>>>CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,DIDHG>IIHIHHCEIIIEB5D@;=H5BIHIHHI<DAIIBDIIFIIH...
24696042,MU069435.1,1567,N,291,>>>>T$T$T$T$T$T$T$T$T$T$T$T$T$T$T$TTTTTTTTTT$T...,DIDHFHGIIIIIDDIII?B=G::B=G=DIHIHIID@BIIGGIIHIH...


In [11]:
import re
toremove = re.compile('[<>$*#]')  
toremove2= re.compile('\^.')  
pileup_35L33G['matches'] = pileup_35L33G['basereads'].apply(lambda x: toremove.sub('', x))
pileup_35L33G['matches'] = pileup_35L33G['matches'].apply(lambda x: toremove2.sub('', x))
pileup_35L33G

,chrom,pos,_ref,count,basereads,quals,matches
0,chr1,3222722,N,124,CCCAAAAAAAAAAAAAAAAAAAAAAAAA^!A^!A^!A^!A^!A^!A...,HCGGHEIHHHIBDDIIIGIG>GIHGDHHDDIAEIEEBIGIBGG8GI...,CCCAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
1,chr1,3222723,N,146,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,E>HIHFIIHHIGDFIII<IHBIIHGDHEB?I@HIEIBIII;>G;GI...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...
2,chr1,3222724,N,155,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,H>GIGHHIHHDG:@IIIBIHDIIHG=HHDDG;IHHH;IIIDG?DIE...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
3,chr1,3222725,N,157,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...,HGDEHHIIHHIG?DIIIEIGDIIHDBBH=DI;IIHBBIII4GD3GI...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...
4,chr1,3222726,N,156,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,HB:IHGIIGGIGDFHII@IFBHGHGBHGCBI@IIHGBIGI;>E@BI...,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...
...,...,...,...,...,...,...,...
24696039,MU069435.1,1564,N,305,>>>>TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,DIDHGHIIGIGIEDIGIHIIG?FBGG@E=:A7G8@HHIHIIDDBII...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...
24696040,MU069435.1,1565,N,297,>>>>AAAAAAAAAAAA$AA$AA$A$AA$AA$A$AAAAAAAAAAAAA...,DIDHBHHIIIGICGIGIEIHDEFBGE8C=1?1G88IHIHIIDGBHI...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
24696041,MU069435.1,1566,N,289,>>>>CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,DIDHG>IIHIHHCEIIIEB5D@;=H5BIHIHHI<DAIIBDIIFIIH...,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...
24696042,MU069435.1,1567,N,291,>>>>T$T$T$T$T$T$T$T$T$T$T$T$T$T$T$TTTTTTTTTT$T...,DIDHFHGIIIIIDDIII?B=G::B=G=DIHIHIID@BIIGGIIHIH...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...


In [12]:
pileup_35L33G[['chrom', 'pos','_ref','matches']]

,chrom,pos,_ref,matches
0,chr1,3222722,N,CCCAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
1,chr1,3222723,N,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...
2,chr1,3222724,N,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
3,chr1,3222725,N,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...
4,chr1,3222726,N,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...
...,...,...,...,...
24696039,MU069435.1,1564,N,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...
24696040,MU069435.1,1565,N,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
24696041,MU069435.1,1566,N,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...
24696042,MU069435.1,1567,N,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...


In [15]:
pileup_35L33G['entropy'] = pileup_35L33G['matches'].map(lambda x : estimate_shannon_entropy(x))
pileup_35L33G['entropy']

0           0.164379
1           0.000000
2           0.000000
3           0.203572
4           1.401164
              ...   
24696039    0.000000
24696040    0.000000
24696041    0.033667
24696042    0.000000
24696043    0.000000
Name: entropy, Length: 24696044, dtype: float64

In [16]:
pileup_35L33G.to_csv("pileup35L33G_filtered.csv", sep=',',index=False)

### 이후 분석은 R로 진행할 예정